In [ ]:
import gc
import tensorflow as tf
gc.collect()
tf.keras.backend.clear_session()


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet101V2
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

# Define dataset path
data_dir = r"C:\Users\Deepak\Desktop\Diabetic-Retinopathy-Detector-using-Ensemble-Learning-main\dataset\train"

# Image data generators with improved augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=8,
    class_mode="categorical",
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=8,
    class_mode="categorical",
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=1,
    class_mode="categorical",
    shuffle=False
)

# Load Pretrained ResNet101V2 model
base_model = ResNet101V2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze initial layers

# Add custom layers
global_avg_pooling = GlobalAveragePooling2D()(base_model.output)
dropout = Dropout(0.5)(global_avg_pooling)
dense = Dense(256, activation='relu')(dropout)
out_layer = Dense(train_generator.num_classes, activation='softmax')(dense)

model = Model(inputs=base_model.input, outputs=out_layer)

# Compile model with Adam optimizer
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0005),
    metrics=['accuracy']
)

# Define callbacks
checkpoint = ModelCheckpoint(
    "best_model.h5", monitor='val_accuracy', save_best_only=True, mode='max', verbose=1
)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=1)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)

# Train the model
history = model.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator,
    callbacks=[checkpoint, early_stopping, lr_scheduler]
)

# Fine-tuning: Unfreeze some layers
base_model.trainable = True
for layer in base_model.layers[:100]:  # Keep first 100 layers frozen
    layer.trainable = False

# Recompile model with lower learning rate
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=1e-5),
    metrics=['accuracy']
)

# Train again
history_finetune = model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    callbacks=[checkpoint, early_stopping, lr_scheduler]
)

# Evaluate on test data
model.load_weights("best_model.h5")
eval_results = model.evaluate(test_generator)
print(f"Test Accuracy: {eval_results[1] * 100:.2f}%")

# Plot training history
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title("Model Accuracy")

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title("Model Loss")

plt.show()


In [ ]:
def predict_image(image_path, model):
    img = PIL.Image.open(image_path).resize((224, 224))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    print(f"Predicted Class: {predicted_class}")
    plt.imshow(img)
    plt.title(f"Predicted: {predicted_class}")
    plt.show()

# Example Test
predict_image(r"C:\Users\Deepak\Desktop\Diabetic-Retinopathy-Detector-using-Ensemble-Learning-main\00a8624548a9.png", model)
